# Industry momentum with traded sector ETFs

In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go
import statsmodels.api as sm
ticker_list = ['IYW','IBB','IHI','SOXX','IGV','ITA','IYR','IYH','IGM','IYE','ICF','IYF','USRT','IHF','IYK','IFRA','IYG','ITB','IEO','IYJ','IDU','IGE','IYM','IAT','IYT','IYC','REZ','REM','IAI','IHE','IAK','IYZ','IEZ','IGN']

# Pull the data from Yahoo
df = pdr(ticker_list, "yahoo", start=2000)
df = df.loc[:,('Adj Close',slice(None))].droplevel('Attributes', axis=1)
df = df.resample('M').last()
df = df.pct_change()
df.index = df.index.to_period("M")
df

Symbols,IYW,IBB,IHI,SOXX,IGV,ITA,IYR,IYH,IGM,IYE,...,IYT,IYC,REZ,REM,IAI,IHE,IAK,IYZ,IEZ,IGN
Date,,,,,,,,,,,,,,,,,,,,,
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06,0.107133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046095,NaN,NaN
2000-07,-0.057080,NaN,NaN,NaN,NaN,NaN,0.063505,-0.034734,NaN,-0.033888,...,NaN,0.003154,NaN,NaN,NaN,NaN,NaN,-0.078683,NaN,NaN
2000-08,0.137438,NaN,NaN,NaN,NaN,NaN,-0.015082,0.025500,NaN,0.117141,...,NaN,0.001048,NaN,NaN,NaN,NaN,NaN,-0.024802,NaN,NaN
2000-09,-0.151667,NaN,NaN,NaN,NaN,NaN,0.031051,0.040956,NaN,0.016292,...,NaN,-0.025130,NaN,NaN,NaN,NaN,NaN,-0.053179,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06,-0.093890,0.008493,-0.072471,-0.178349,-0.056806,-0.026880,-0.068911,-0.024183,-0.099943,-0.164164,...,-0.083060,-0.106547,-0.058776,-0.108045,-0.087120,-0.010770,-0.059700,-0.075285,-0.211944,-0.070306
2022-07,0.114947,0.054833,0.065424,0.165098,0.095323,0.046688,0.088606,0.034434,0.129250,0.111490,...,0.093988,0.154565,0.075634,0.144727,0.105514,-0.005844,-0.008437,0.016315,0.032561,0.184952
2022-08,-0.060242,-0.021115,-0.059732,-0.092407,-0.047509,-0.020135,-0.058624,-0.061363,-0.057274,0.034222,...,-0.018505,-0.028698,-0.054161,-0.071545,-0.001077,-0.075526,0.012277,-0.030932,-0.003032,0.011036


In [3]:
# Estimation window in months for past average returns
WINDOW = 12

# Fill missings with cross-sectional average
for c in df.columns:
    df[c]=df[c].fillna(df.mean(axis=1))

# Calculate rolling average returns
avgs = df.rolling(WINDOW).mean()  # this average is inclusive of the return in a given row
avgs = avgs.iloc[WINDOW:]


In [4]:
# Split into portfolios (not enough to do more than two portfolios each period)
ports = avgs.apply(lambda x: pd.qcut(x, 2,labels=False), axis=1)
ports

Symbols,IYW,IBB,IHI,SOXX,IGV,ITA,IYR,IYH,IGM,IYE,...,IYT,IYC,REZ,REM,IAI,IHE,IAK,IYZ,IEZ,IGN
Date,,,,,,,,,,,,,,,,,,,,,
2001-05,0,1,0,1,0,1,1,0,1,1,...,0,1,0,0,0,0,0,0,1,1
2001-06,0,1,0,1,0,1,1,0,1,1,...,0,1,0,0,0,0,0,0,1,1
2001-07,0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,0,0,0,1,1
2001-08,0,1,0,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,0,0
2001-09,0,0,0,0,0,1,1,1,0,1,...,0,1,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06,0,0,0,0,0,1,1,1,0,1,...,0,0,1,0,0,1,1,0,1,0
2022-07,0,0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,1,1,0,1,1
2022-08,0,0,0,0,0,1,0,0,0,1,...,1,0,1,0,0,1,1,0,1,1


In [5]:

# Portfolio of the most recent winners
hi = pd.DataFrame(dtype=float, columns=ports.columns, index=ports.index)
for c in ports.columns:
    hi[c] = (ports[c]==1)*1.0
hi=hi.div(hi.sum(axis=1), axis=0)  # Normalize so weights sum to 1

# Portfolio of the most recent losers
lo = pd.DataFrame(dtype=float, columns=ports.columns, index=ports.index)
for c in ports.columns:
    lo[c] = (ports[c]==0)*1.0
lo=lo.div(lo.sum(axis=1), axis=0)  # Normalize so weights sum to 1

#  Multiply weights by returns in next month
rets = pd.DataFrame(dtype=float,columns=['hi','lo'],index=hi.index[1:])
for d in rets.index:
    rets.loc[d,'hi'] = hi.loc[d-1] @ df.loc[d]
    rets.loc[d,'lo'] = lo.loc[d-1] @ df.loc[d]
rets.index = rets.index.to_timestamp('M')
rets

,hi,lo
Date,,
2001-06-30,-0.010227,-0.013423
2001-07-31,-0.029662,-0.020879
2001-08-31,-0.043716,-0.058961
2001-09-30,-0.076862,-0.140701
2001-10-31,0.026658,0.090535
...,...,...
2022-06-30,-0.097393,-0.083420
2022-07-31,0.061494,0.107849
2022-08-31,-0.016220,-0.043322


In [9]:
# Plot cumulative returns
string =  "Strategy: Buy Past Winners<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_hi  = go.Scatter(x=rets.index, y=(1+rets.hi).cumprod(), mode="lines", name='Buy Winners', hovertemplate=string)
string =  "Strategy: Buy Past Losers<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_lo= go.Scatter(x=rets.index, y=(1+rets.lo).cumprod(), mode="lines", name='Sell Losers',hovertemplate=string)
fig = go.Figure()
fig.add_trace(trace_hi)
fig.add_trace(trace_lo)
fig.update_yaxes(title="FV of $1")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
fig.show()

In [10]:
# Plot cumulative returns of long-short strategy
string =  "Strategy: Buy Winners and Short Losers<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace  = go.Scatter(x=rets.index, y=(1+rets.hi-rets.lo).cumprod(), mode="lines", name='Long-Short', hovertemplate=string)
fig = go.Figure()
fig.add_trace(trace)
fig.update_yaxes(type="log",title='FV of $1')
fig.update_layout(title='Industry Momentum')
fig.show()

### Performance Statistics

In [11]:
# Portfolio alpha and beta
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
rets.index=rets.index.to_period("M")
df = rets.join(ff3[['Mkt-RF','RF']])
df['hi-rf'] = df['hi']-df['RF']
df['lo-rf'] = df['lo']-df['RF']
df['hi-lo'] = df['hi']-df['lo']

In [22]:
def mkt_model(varname):
    mm = sm.OLS(df[varname], sm.add_constant(df['Mkt-RF']),missing='drop').fit()
    alpha,beta = mm.params
    print(f'Annualized alpha of long portfolio: {alpha*12: .2%}\n')
    print(mm.summary())

In [23]:
mkt_model('hi-rf')

Annualized alpha of long portfolio:  0.28%

                            OLS Regression Results                            
Dep. Variable:                  hi-rf   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     2103.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):          7.17e-125
Time:                        15:19:03   Log-Likelihood:                 704.99
No. Observations:                 256   AIC:                            -1406.
Df Residuals:                     254   BIC:                            -1399.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [24]:
mkt_model('lo-rf')

Annualized alpha of long portfolio: -1.86%

                            OLS Regression Results                            
Dep. Variable:                  lo-rf   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     1993.
Date:                Thu, 27 Oct 2022   Prob (F-statistic):          3.13e-122
Time:                        15:19:14   Log-Likelihood:                 652.85
No. Observations:                 256   AIC:                            -1302.
Df Residuals:                     254   BIC:                            -1295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [25]:
mkt_model('hi-lo')

Annualized alpha of long portfolio:  2.14%

                            OLS Regression Results                            
Dep. Variable:                  hi-lo   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     27.33
Date:                Thu, 27 Oct 2022   Prob (F-statistic):           3.58e-07
Time:                        15:19:33   Log-Likelihood:                 569.74
No. Observations:                 256   AIC:                            -1135.
Df Residuals:                     254   BIC:                            -1128.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const   